In [ ]:
from collections import OrderedDict
import pandas as pd

from nnood.data.dataset_conversion.convert_mvtec import OBJECTS, TEXTURES
from nnood.utils.file_operations import load_results_json
import numpy as np

def get_results(dataset, task, plans_identifier):
    result_metrics = load_results_json(dataset, task, plans_identifier)['results']
    return result_metrics

In [ ]:

tasks = ['FPI', 'CutPaste']
metrics = ['AP score', 'AUROC']

In [ ]:
results = OrderedDict()

for m in metrics:
    results[m] = []

for class_name in OBJECTS + TEXTURES:
    
    if class_name is TEXTURES[0]:
        # Add objects average
        for m in metrics:
            all_object_results = results[m]
            results[m].append(np.mean(all_object_results, axis=0))
    
    for m in metrics:
        results[m].append([])
    
    for t in tasks:
        metric_results = get_results('mvtec_ad_' + class_name, t)

        for m in metrics:
            results[m][-1].append(metric_results[m])

for m in metrics:
    # Add texture average
    all_texture_results = results[m][-len(TEXTURES):]
    
    results[m].append(np.mean(all_texture_results, axis=0))
    
    # Add total average
    all_nonaverage_results = np.concatenate((results[m][:len(OBJECTS)], results[m][len(OBJECTS) + 1: -1]))
    
    assert all_nonaverage_results.shape[0] == (len(OBJECTS) + len(TEXTURES)), f'Shape: {all_nonaverage_results.shape}'
    results[m].append(np.mean(all_nonaverage_results, axis=0))


                
    

In [ ]:
miindex = pd.MultiIndex.from_tuples(
    [('Object', obj) for obj in OBJECTS + ['Average']] + [('Texture', txtr) for txtr in TEXTURES + ['Average']] + [('Total', 'Average')])
dataframes = OrderedDict()

for m in metrics:
    dataframes[m] = pd.DataFrame(results[m], index=miindex, columns=tasks)


In [ ]:
dataframes['AUROC']

In [ ]:
auroc_avgs = dataframes['AUROC'].loc['Total', 'Average']
